In [20]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import ipywidgets as widgets
from IPython.display import display, clear_output

class ConcentricInteractiveEvolution:
    def __init__(self):
        self.num_points = 23  # Total points (1 center + 22 on circles)
        self.num_layouts = 9
        self.canvas_size = 400
        self.margin = 50

        # Define the radii for the three concentric circles
        self.center = np.array([self.canvas_size/2, self.canvas_size/2])
        self.radii = [50, 100, 150]  # Inner, middle, and outer circle radii

        self.point_distributions = [

            [6, 8, 8],
            [7, 7, 8],
            [8, 7, 7],
            [6, 7, 9],
            [7, 8, 7],
        ]

        self.weights = {
            'symmetry': 1.0,
            'spacing': 1.0,
            'spiral': 1.0,
            'uniqueness': 1.0,
            'local_patterns': 1.0,
            'star_pattern': 1.5
        }
        self.selection_history = []
        self.output = widgets.Output()
    def evaluate_23_unique_pattern(self, points):
        center = np.mean(points, axis=0)
        distances = np.linalg.norm(points - center, axis=1)
        sorted_indices = np.argsort(distances)

        inner_points = points[sorted_indices[:11]]
        outer_points = points[sorted_indices[11:]]

        inner_std = np.std(np.linalg.norm(inner_points - center, axis=1))
        outer_std = np.std(np.linalg.norm(outer_points - center, axis=1))

        return -(inner_std + outer_std)
    def run_evolution_steps(self, population, selected_layout):
        current_best = selected_layout
        current_population = population

        print("\ninit...")
        for step in range(self.evolution_steps):
            # next gen
            current_population = self.evolve(current_best, current_population)

            scores = [self.evaluate_layout(p) for p in current_population]
            best_idx = np.argmax(scores)
            current_best = current_population[best_idx]

            print(f"involve {step+1}/{self.evolution_steps}")
            print(f"best score: {scores[best_idx]:.2f}")

        return current_population
    def evaluate_star_pattern(self, points):
        center = np.mean(points, axis=0)

        distances = np.linalg.norm(points - center, axis=1)
        angles = np.arctan2(points[:, 1] - center[1], points[:, 0] - center[0])


        median_dist = np.median(distances)
        inner_points = points[distances <= median_dist]
        outer_points = points[distances > median_dist]

        score = 0


        sorted_angles = np.sort(angles)
        angle_diffs = np.diff(sorted_angles)
        angle_regularity = -np.std(angle_diffs)
        score += angle_regularity * 2

        sorted_dist = np.sort(distances)
        dist_diffs = np.diff(sorted_dist)
        dist_pattern = -np.std(dist_diffs)
        score += dist_pattern

        radial_score = 0
        for outer_point in outer_points:
            inner_distances = np.linalg.norm(inner_points - outer_point, axis=1)
            nearest_inner = np.argmin(inner_distances)

            outer_angle = np.arctan2(outer_point[1] - center[1],
                                    outer_point[0] - center[0])
            inner_angle = np.arctan2(inner_points[nearest_inner][1] - center[1],
                                    inner_points[nearest_inner][0] - center[0])
            angle_diff = np.abs(outer_angle - inner_angle)
            radial_score -= min(angle_diff, np.pi - angle_diff)

        score += radial_score / len(outer_points)
        symmetry_score = 0
        for i, point in enumerate(points):
            point_angle = np.arctan2(point[1] - center[1],
                                    point[0] - center[0])
            point_dist = np.linalg.norm(point - center)
            opposite_angle = point_angle + np.pi
            for other_point in points[i+1:]:
                other_angle = np.arctan2(other_point[1] - center[1],
                                      other_point[0] - center[0])
                other_dist = np.linalg.norm(other_point - center)
                angle_match = min(abs(other_angle - opposite_angle),
                                abs(other_angle - opposite_angle + 2*np.pi),
                                abs(other_angle - opposite_angle - 2*np.pi))

                if angle_match < 0.1 and abs(point_dist - other_dist) < 10:
                    symmetry_score += 1

        score += symmetry_score

        return score

    def evaluate_local_patterns(self, points):
        score = 0
        for p in points:
            distances = np.linalg.norm(points - p, axis=1)
            nearest = np.partition(distances, 6)[1:6]
            score -= np.std(nearest)
        return score

    def generate_circle_points(self, radius, num_points, noise_factor=0.1):
        base_angles = np.linspace(0, 2*np.pi, num_points, endpoint=False)
        start_offset = np.random.uniform(0, 2*np.pi)
        angles = base_angles + start_offset
        angles += np.random.normal(0, noise_factor, num_points)
        x = self.center[0] + radius * np.cos(angles)
        y = self.center[1] + radius * np.sin(angles)

        return np.column_stack((x, y))
    def run_interactive(self):
        population = self.create_population()
        generation = 0

        dropdown = widgets.Dropdown(
            options=[(f"Layout {i+1}", i) for i in range(self.num_layouts)],
            description="seletc layout:",
            style={'description_width': 'initial'}
        )

        steps_slider = widgets.IntSlider(
            value=5,
            min=1,
            max=20,
            step=1,
            description='mutate num:',
            style={'description_width': 'initial'}
        )

        next_button = widgets.Button(
            description="select",
            button_style="success"
        )
        def on_button_click(b):
            nonlocal population, generation
            selected_idx = dropdown.value
            self.evolution_steps = steps_slider.value
            selected_layout = population[selected_idx]

            self.selection_history.append({
                'generation': generation,
                'selected_features': self.calculate_features(selected_layout),
                'weights': self.weights.copy()
            })

            with self.output:
                population = self.run_evolution_steps(population, selected_layout)
                generation += self.evolution_steps
                print(f"\nfinish {self.evolution_steps}")
                print(f"now: {generation}")

            # next gen
            self.display_layouts(population)

        next_button.on_click(on_button_click)

        # control
        controls = widgets.VBox([
            widgets.HBox([dropdown, steps_slider]),
            next_button,
            self.output
        ])
        display(controls)
        self.display_layouts(population)

    def generate_layout(self):

        distribution = self.point_distributions[np.random.randint(len(self.point_distributions))]

        points = [self.center]

        for radius, num_points in zip(self.radii, distribution):
            circle_points = self.generate_circle_points(radius, num_points)
            points.extend(circle_points)

        return np.array(points)

    def calculate_features(self, points):
        features = {}
        center = points[0]  # Use the fixed center point
        distances_to_center = np.linalg.norm(points[1:] - center, axis=1)
        features['symmetry'] = -np.std(distances_to_center)

        distances = pdist(points)
        features['spacing'] = -np.std(distances)

        angles = np.arctan2(points[1:, 1] - center[1], points[1:, 0] - center[0])
        sorted_angles = np.sort(angles)
        angle_diffs = np.diff(sorted_angles)
        features['spiral'] = -np.std(angle_diffs)

        features['uniqueness'] = self.evaluate_23_unique_pattern(points)
        features['local_patterns'] = self.evaluate_local_patterns(points)
        features['star_pattern'] = self.evaluate_star_pattern(points)

        return features

    def mutate(self, points, mutation_rate=0.2, mutation_strength=0.2):

        mutated_points = [points[0]]
        remaining_points = points[1:]
        distances = np.linalg.norm(remaining_points - self.center, axis=1)

        inner_mask = distances < (self.radii[0] + self.radii[1])/2
        outer_mask = distances > (self.radii[1] + self.radii[2])/2
        middle_mask = ~(inner_mask | outer_mask)

        for mask, radius in zip([inner_mask, middle_mask, outer_mask], self.radii):
            circle_points = remaining_points[mask]
            for point in circle_points:
                if np.random.random() < mutation_rate:
                    # Calculate current angle
                    current_angle = np.arctan2(point[1] - self.center[1],
                                             point[0] - self.center[0])
                    new_angle = current_angle + np.random.normal(0, mutation_strength)

                    new_point = self.center + radius * np.array([
                        np.cos(new_angle),
                        np.sin(new_angle)
                    ])

                    mutated_points.append(new_point)
                else:
                    mutated_points.append(point)

        return np.array(mutated_points)



    def evaluate_layout(self, points):

        features = self.calculate_features(points)
        return sum(self.weights[k] * features[k] for k in features)

    def crossover(self, layout1, layout2):

        new_layout = [self.center]
        alpha = np.random.random()
        remaining_points = alpha * layout1[1:] + (1 - alpha) * layout2[1:]
        new_layout.extend(remaining_points)

        return np.array(new_layout)

    def create_population(self):
        return [self.generate_layout() for _ in range(self.num_layouts)]

    def display_layouts(self, population):

        with self.output:
            clear_output(wait=True)
            fig = plt.figure(figsize=(5, 5))
            for i, layout in enumerate(population):
                plt.subplot(3, 3, i + 1)

                plt.scatter(layout[0, 0], layout[0, 1], c='red', s=5)

                plt.scatter(layout[1:, 0], layout[1:, 1], c='blue', s=5)
                score = self.evaluate_layout(layout)
                plt.title(f'Layout {i+1}\nScore: {score:.2f}')
                plt.xticks([])
                plt.yticks([])
            plt.tight_layout()
            plt.show()



    def evolve(self, selected_layout, population):

        new_population = [selected_layout]

        for _ in range(5):
            mutated = self.mutate(selected_layout.copy())
            new_population.append(mutated)

        # Add some completely new layouts
        for _ in range(2):
            new_layout = self.generate_layout()
            new_population.append(new_layout)

        for _ in range(1):
            other_layout = population[np.random.randint(len(population))]
            hybrid = self.crossover(selected_layout, other_layout)


            hybrid_points = [hybrid[0]]  # Keep center
            remaining_points = hybrid[1:]
            distances = np.linalg.norm(remaining_points - self.center, axis=1)

            for point in remaining_points:
                dist = np.linalg.norm(point - self.center)
                closest_radius_idx = np.argmin(np.abs(np.array(self.radii) - dist))
                correct_radius = self.radii[closest_radius_idx]

                # Calculate angle and create corrected point
                angle = np.arctan2(point[1] - self.center[1], point[0] - self.center[0])
                corrected_point = self.center + correct_radius * np.array([
                    np.cos(angle),
                    np.sin(angle)
                ])
                hybrid_points.append(corrected_point)

            new_population.append(np.array(hybrid_points))

        return new_population


if __name__ == "__main__":
    evolution = ConcentricInteractiveEvolution()
    evolution.run_interactive()